## 实验介绍

### 1.实验内容

本实验介绍SVD应用协同推荐。并使用该算法实现餐馆推荐引擎。

### 2.实验目标

通过本实验掌握SVD算法。

### 3.实验知识点

* SVD算法

### 4.实验环境

* python 3.6.5

### 5.预备知识

* Python编程基础

## 【实验】餐馆菜肴推荐引擎 
本节实验我们来实现一个基于SVD的菜肴推荐引擎。该引擎关注的是餐馆食物的推荐。由于协同推荐是基于用户历史数据来挖掘的，那么我们需要先寻找用户没有尝过的菜肴，通过尝过的菜肴评分来预测这道菜肴的可能得分。接下来，通过SVD来减少特征空间并提高推荐的效果。

我们需要明确的是，推荐系统的工作过程为，给定一个用户，系统会为此用户返回N个最好的推荐菜。为了实现这一点，则需要我们做到：

    (1) 寻找用户没有评级的菜肴，即在用户－物品矩阵中的0值；
    (2) 在用户没有评级的所有物品中，对每个物品预计可能的评级分数。这就是说，我们认为用户可能会对物品的打分（这就是相似度计算的初衷）；
    (3) 对这些物品的评分从高到低进行排序，返回前N个物品。

## 【实验】推荐未尝过的菜 
### 1.推荐未尝过的菜
根据推荐系统的工作过程，我们需要定义功能函数来实现推荐和估计评分。由于对物品的估计评分是一个重复操作，我们单独用一个函数来实现。

    standEst -计算给定相似度计算方法的条件下，用户对物品的估计评分值。
    recommend -寻找未评价的物品，对物品评分进行排序

添加如下代码：

In [1]:
from numpy import *
from numpy import linalg as la
def loadExData1():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]
"""
函数说明：给定相似度计算方式，评估物品得分
parameters:
    dataMat -数据矩阵
    user -用户编号
    simMeas -相似度计算方法引用
    item -物品编号
return:
    用户对该物品的预估评分
"""
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]   #行为用户，列为物品。得到物品个数
    simTotal = 0.0;ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]    #记录用户对物品j的评分
        if userRating == 0: continue    #用户未对该物品评分，继续
        overLap = nonzero(logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0] 
        #print(dataMat[:,item].A)
        #print(logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))
        #print(nonzero(logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0)))
        if len(overLap) == 0: similarity = 0    #两个物品没有重合元素，相似度为0
        else:
            similarity = simMeas(dataMat[overLap,item],dataMat[overLap,j])  #对重合元素进行相似度计算
        print("物品%d和物品%d的相似度为%f" %(item,j,similarity))
        simTotal += similarity  #评分总和
        ratSimTotal +=similarity*userRating #相似度和当前用户评分的乘积
    if simTotal == 0 : return 0.0   
    else: return ratSimTotal/simTotal   #对数据归一，使得评分值在0-5之间

"""
函数说明：相似度计算函数(余弦相似度)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def cosSim(inA, inB):
    num = float(inA.T*inB)  #计算分子
    denom = la.norm(inA)*la.norm(inB)   #计算分母
    return 0.5+0.5*(num/denom)  #归一化

"""
函数说明：推荐函数
parameters:
    dataMat -数据矩阵
    user -用户编号
    N -产生推荐结果的个数
    simMeas -相似度计算方法
    estMethod -评估函数
return:
    返回N个推荐结果
"""
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod = standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]     #找到该用户还未评分的物品
    if len(unratedItems) == 0: return "你吃过的东西太多啦"
    itemScores = []
    for item in unratedItems:   
        estimatedScore = estMethod(dataMat,user,simMeas,item)   #计算该物品的预估得分
        itemScores.append((item,estimatedScore))    #存储到list
    return sorted(itemScores,key=lambda jj: jj[1],reverse=True)[:N] #将得分排序，返回前N个

"""
函数说明：相似度计算函数(欧氏距离)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def ecludSim(inA, inB):
    return 1.0/(1.0+la.norm(inA-inB))
"""
函数说明：相似度计算函数(皮尔逊相关系数)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def pearsSim(inA, inB):
    if len(inA)<3: return 1.0   #是否存在三个或更多的点。两个向量是完全相关的，返回1
    #print(corrcoef(inA,inB,rowvar=0))
    return 0.5+0.5*corrcoef(inA,inB,rowvar=0)[0][1]     #将数据归一化到0到1之间

if __name__ == '__main__':
    #测试推荐
    myMat = mat(loadExData1())
    myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
    myMat[3,3]=2
    print("修改后的矩阵：\n",myMat)
    print("余弦计算相似度推荐：\n",recommend(myMat,2))
    print("欧式相似度推荐：\n",recommend(myMat,2,simMeas=ecludSim))
    print("皮尔逊计算相似度推荐：\n",recommend(myMat,2,simMeas=pearsSim))

修改后的矩阵：
 [[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]
物品1和物品0的相似度为1.000000
物品1和物品3的相似度为0.928746
物品1和物品4的相似度为1.000000
物品2和物品0的相似度为1.000000
物品2和物品3的相似度为1.000000
物品2和物品4的相似度为0.000000
余弦计算相似度推荐：
 [(2, 2.5), (1, 2.0243290220056256)]
物品1和物品0的相似度为1.000000
物品1和物品3的相似度为0.309017
物品1和物品4的相似度为0.333333
物品2和物品0的相似度为1.000000
物品2和物品3的相似度为0.500000
物品2和物品4的相似度为0.000000
欧式相似度推荐：
 [(2, 3.0), (1, 2.8266504712098603)]
物品1和物品0的相似度为1.000000
物品1和物品3的相似度为1.000000
物品1和物品4的相似度为1.000000
物品2和物品0的相似度为1.000000
物品2和物品3的相似度为1.000000
物品2和物品4的相似度为0.000000
皮尔逊计算相似度推荐：
 [(2, 2.5), (1, 2.0)]


表明了用户2（由于我们从0开始计数，因此这对应了矩阵的第3行）对物品2的预测评分值为2.5，对物品1的预测评分值为2.05。这个例子给出了如何利用基于物品相似度和多个相似度计算方法来进行推荐的过程，下面我们介绍如何将SVD应用于推荐。

## 【实验】使用SVD改进推荐效果 
### 2.使用SVD改进推荐效果
实际的数据集会比我们用于展示recommend()函数功能的myMat矩阵稀疏得多。我们将使用下图矩阵中的数据来进行推荐。
![](svd_res/1_svd_res.png)
下面我们来计算该矩阵的SVD来了解到底需要多少维特征(多少个奇异值)。

In [2]:
def loadExData2():
    return[[2, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5],
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 3, 0, 0, 2, 2, 0, 0],
           [5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
           [4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5],
           [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
           [0, 0, 0, 3, 0, 0, 0, 0, 4, 5, 0],
           [1, 1, 2, 1, 1, 2, 1, 0, 4, 5, 0]]

if __name__ == '__main__':
    u,sigma,vt = la.svd(mat(loadExData2()))
    print("奇异值矩阵为:\n",sigma)
    #计算总能量的90%
    sig2 = sigma**2
    parSig = sum(sig2)*0.9
    print("90%的能量为：\n",parSig)
    energy = 0.0
    while(energy < parSig):
        i = int(input("请输入所需奇异值个数："))
        energy = sum(sig2[:i])
        print("%d个元素所包含的能量为%f"%(i,energy))

奇异值矩阵为:
 [1.34442785e+01 1.18146312e+01 8.17058402e+00 6.92696870e+00
 5.32673162e+00 3.94953045e+00 2.87742807e+00 2.35058281e+00
 2.19398990e+00 5.77270855e-01 3.26723859e-16]
90%的能量为：
 448.2000000000003
请输入所需奇异值个数：5
5个元素所包含的能量为463.449544


运行后我们可以看到，前4个元素所包含的能量已经足以到总能量的90%。也就是说我们可以将一个11维的矩阵转换成一个4维的矩阵。

这里我们给出一个基于SVD的评分估计函数，与之前的standEst不同的是，这里对数据集进行了SVD分解。

In [3]:
"""
函数说明：基于SVD的评分估计
parameters:
    dataMat -数据矩阵
    user -用户编号
    simMeas -相似度计算方法
    item -物品编号
return:
    物品的估计评分
"""
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]   #获取物品个数
    simTotal = 0.0; ratSimTotal = 0.0
    u,sigma,vt = la.svd(dataMat)    #对数据矩阵奇异值分解
    sig4 = mat(eye(4)*sigma[:4])    #只利用包含了90%能量值的奇异值      
    xformedItems = dataMat.T * u[:,:4] *sig4.I  #利用u矩阵将物品转换到低维空间
    for j in range(n):
        userRating = dataMat[user,j]    #得到用户对该物品的评分
        if userRating == 0 or j==item:continue   #如果评分为0或者物品与比较物品相同，则跳过
        similarity = simMeas(xformedItems[item,:].T,xformedItems[j,:].T)    #转换为列向量，并计算相似度
        print("物品%d和物品%d的相似度为%f" %(item,j,similarity))
        simTotal += similarity  
        ratSimTotal += similarity*userRating  
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal   
    
if __name__ == '__main__':
    #测试svd推荐
    myMat1 = mat(loadExData2())
    print("使用svdEst,cosSim的推荐结果：\n",recommend(myMat1,1,estMethod=svdEst))
    print("使用svdEst,pearsSim的推荐结果：\n",recommend(myMat1,1,estMethod=svdEst,simMeas=pearsSim))
    print("使用standEst,pearsSim的推荐结果：\n",recommend(myMat1,1,simMeas=pearsSim))

物品0和物品3的相似度为0.486344
物品0和物品10的相似度为0.580563
物品1和物品3的相似度为0.398911
物品1和物品10的相似度为0.345523
物品2和物品3的相似度为0.423278
物品2和物品10的相似度为0.563631
物品4和物品3的相似度为0.934678
物品4和物品10的相似度为0.412897
物品5和物品3的相似度为0.747099
物品5和物品10的相似度为0.941154
物品6和物品3的相似度为0.214751
物品6和物品10的相似度为0.504010
物品7和物品3的相似度为0.664213
物品7和物品10的相似度为0.308751
物品8和物品3的相似度为0.967466
物品8和物品10的相似度为0.451672
物品9和物品3的相似度为0.619705
物品9和物品10的相似度为0.490821
使用svdEst,cosSim的推荐结果：
 [(6, 3.8048799321710414), (2, 3.2844276897522917), (5, 3.2298885106968034)]
物品0和物品3的相似度为0.519340
物品0和物品10的相似度为0.568226
物品1和物品3的相似度为0.332656
物品1和物品10的相似度为0.343553
物品2和物品3的相似度为0.447018
物品2和物品10的相似度为0.541048
物品4和物品3的相似度为0.888466
物品4和物品10的相似度为0.609317
物品5和物品3的相似度为0.856513
物品5和物品10的相似度为0.991215
物品6和物品3的相似度为0.113273
物品6和物品10的相似度为0.511771
物品7和物品3的相似度为0.499727
物品7和物品10的相似度为0.413594
物品8和物品3的相似度为0.971012
物品8和物品10的相似度为0.847361
物品9和物品3的相似度为0.318460
物品9和物品10的相似度为0.753570
使用svdEst,pearsSim的推荐结果：
 [(6, 4.275103895098489), (9, 3.8117492707221783), (2, 3.1903310379709113)]
物品0和物品3的相似度为1.000000
物品0和物品

我们可以看到不同的估计评分函数下，不同的相似度计算下的推荐结果。但在大型推荐系统中，SVD分解在效率上会降低程序的速度。

## 实验总结

本节我们介绍了SVD通过在低维空间下计算相似度，提高了推荐引擎的效果。介绍了如何应用该算法，您应该能达到以下两个目标：

1. 掌握SVD算法。

2. 学会实现相应算法。

##  参考文献及延伸阅读 

### 参考资料：

1.哈林顿，李锐. 机器学习实战 : Machine learning in action[M]. 人民邮电出版社, 2013.  
2.周志华. 机器学习:Machine learning[M]. 清华大学出版社, 2016.

### 延伸阅读：

1.李航. 统计学习方法[M]. 清华大学出版社, 2012.